### Summary

작성일 : 19/08/13  
  
<목적>  
client_id를 추출할 수 있는 새로운 조합 찾기

<분석 배경>  
지금까지 client_id를 TransactionAmt(거래액), Card1, Deviceinfo가 동일한 건으로 묶어서 봐왔음.  
이 방법은 같은 고객이 다른 카드를 쓸 수도 있고, 거래액도 각기 다를 수 있다는 점에서 한계가 있음

<진행 방법>  
V 변수 중 count가 증가하는 변수(ex 1, 2, 3, ...)와 transactionAmt를 더해서 증가하는 변수(ex 가격이 20이라면 20, 40, 60 , 80 ...)  
를 통해 같은 유저로 추정되는 사람을 묶기

<향후 계획>  
같은 유저로 판명된 경우 해당 유저들의 특성을 파악하고, 의미있는 행동패턴이나 차이를 보이는지 확인하기

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc, datetime, time
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
from plotnine import *
import warnings
warnings.filterwarnings("ignore")

In [2]:
train_trans = pd.read_csv("/Users/jeong/downloads/IEEE/input/train_transaction.csv")
train_idf = pd.read_csv("/Users/jeong/downloads/IEEE/input/train_identity.csv")

test_trans = pd.read_csv("/Users/jeong/downloads/IEEE/input/test_transaction.csv")
test_idf = pd.read_csv("/Users/jeong/downloads/IEEE/input/test_identity.csv")

train_trans = pd.merge(train_trans, train_idf, on = 'TransactionID', how = 'left')
test_trans = pd.merge(test_trans, test_idf, on = 'TransactionID', how = 'left')

In [3]:
# account_make_day 만들기
# 현우님 커널 코드 퍼왔습니다.

start_date = datetime.datetime.strptime('2017.11.30', '%Y.%m.%d')
train_trans['timeblock'] = train_trans['TransactionDT'].apply(lambda x: datetime.timedelta(seconds = x) + start_date ) 
test_trans['timeblock'] = test_trans['TransactionDT'].apply(lambda x: datetime.timedelta(seconds = x) + start_date ) 

tb = train_trans['timeblock']
train_trans.drop('timeblock', 1, inplace=True)
train_trans.insert(0, 'timeblock', tb)

tb = test_trans['timeblock']
test_trans.drop('timeblock', 1, inplace=True)
test_trans.insert(0, 'timeblock', tb)

def account_start_date(val):
    if np.isnan(val) :
        return np.NaN
    else:
        days=  int( str(val).split('.')[0])
        return pd.Timedelta( str(days) +' days')

train_trans['account_start_day'] = train_trans['D8'].apply(account_start_date)
test_trans['account_start_day'] = test_trans['D8'].apply(account_start_date)

train_trans['account_make_date'] = (train_trans['timeblock'] - train_trans['account_start_day']).dt.date
test_trans['account_make_date'] = (test_trans['timeblock'] - test_trans['account_start_day']).dt.date

In [4]:
ex = train_trans[train_trans['TransactionAmt'] == 106][['TransactionDT','TransactionAmt','V95','V126']]
ex.sort_values('V95', ascending = False, inplace = True)
ex2 = ex.iloc[:7,:]
ex2.sort_values('V95')

,TransactionDT,TransactionAmt,V95,V126
562344,14908261,106.0,874.0,94265.0
562346,14908270,106.0,875.0,94371.0
562347,14908275,106.0,876.0,94477.0
562348,14908285,106.0,877.0,94583.0
562350,14908290,106.0,878.0,94689.0
562352,14908301,106.0,879.0,94795.0
562353,14908306,106.0,880.0,94901.0


V95의 경우 count가 1씩 늘어나고, V126의 경우 이전 TransactionAmt까지의 누적 합을 나타냄  

In [5]:
# 1. 빈 데이터 프레임 생성
temp = pd.DataFrame()

# 처음 진행할 값을 정함
train_trans2 = train_trans
train_trans = train_trans[['V95', 'V126','TransactionAmt','TransactionDT']]
temp_n = train_trans[train_trans['V95'] == train_trans['V95'].max()]  
temp_n = temp_n.iloc[[0]] 
temp_n


,V95,V126,TransactionAmt,TransactionDT
562353,880.0,94901.0,106.0,14908306


In [6]:
a = 0
while True :
    print(float(temp_n['V95']))
    temp_n_1 = train_trans[train_trans['V95'] == float(temp_n['V95']-1)] # 여러개일수 있음
    if (temp_n_1.shape[0] == 0) | (float(temp_n['V95']-1) == 600) | (float(temp_n['V95']) == a)  :
        break
    a = float(temp_n['V95'])    
    for i in range(temp_n_1.shape[0]):
        te = temp_n_1.iloc[[i]]
        if (temp_n['V126'].values - te['V126'].values)== te['TransactionAmt'].values :
            temp = temp.append(temp_n) # 삽입
            train_trans.drop(train_trans.index[temp_n.index], inplace = True) #삭제
            temp_n = te  

880.0
879.0
878.0
877.0
876.0
875.0
874.0
873.0
872.0
871.0
870.0
869.0
868.0
867.0
866.0
865.0
864.0
863.0
862.0
861.0
860.0
859.0
858.0
857.0
856.0
855.0
854.0
853.0
852.0
851.0
850.0
849.0
848.0
847.0
846.0
845.0
844.0
843.0
842.0
841.0
840.0
839.0
838.0
837.0
836.0
835.0
834.0
833.0
832.0
831.0
830.0
829.0
828.0
827.0
826.0
825.0
824.0
823.0
822.0
821.0
820.0
819.0
818.0
817.0
816.0
815.0
814.0
813.0
812.0
811.0
810.0
809.0
808.0
807.0
806.0
805.0
804.0
803.0
802.0
801.0
800.0
799.0
798.0
797.0
796.0
795.0
794.0
793.0
792.0
791.0
790.0
789.0
788.0
787.0
786.0
785.0
784.0
783.0
782.0
781.0
780.0
779.0
778.0
777.0
776.0
775.0
774.0
773.0
772.0
771.0
770.0
769.0
768.0
767.0
766.0
765.0
764.0
763.0
762.0
761.0
760.0
759.0
758.0
757.0
756.0
755.0
754.0
753.0
752.0
751.0
750.0
749.0
748.0
747.0
746.0
745.0
744.0
743.0
742.0
741.0
740.0
739.0
738.0
737.0
736.0
735.0
734.0
733.0
732.0
731.0
730.0
729.0
728.0
727.0
726.0
725.0
724.0
723.0
722.0
721.0
720.0
719.0
718.0
717.0
716.0
715.0
714.

In [7]:
temp.sort_values('V95')

,V95,V126,TransactionAmt,TransactionDT
561115,602.0,68863.0,125.0,14859250
561116,603.0,68988.0,125.0,14859258
561117,604.0,69113.0,125.0,14859267
561118,605.0,69238.0,125.0,14859271
561119,606.0,69363.0,125.0,14859280
561121,607.0,69488.0,125.0,14859289
561122,608.0,69613.0,125.0,14859301
561123,609.0,69738.0,125.0,14859306
561125,610.0,69863.0,125.0,14859324
561126,611.0,69988.0,125.0,14859328


V95, V126으로 user를 추출할 수 있음을 확인